In [1]:
import ujson as json

with open("news.json", "r") as f:
    news_list = json.load(f)

In [ ]:
print(news_list[0]["title"])
print(news_list[0]["text"])

In [ ]:
!pip install --upgrade kiwipiepy
!pip install --upgrade wordcloud
!pip install --upgrade tqdm

In [10]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords

kiwi = Kiwi(model_type="sbg")
stopwords = Stopwords()

In [ ]:
# 태그: https://github.com/bab2min/kiwipiepy#%ED%92%88%EC%82%AC-%ED%83%9C%EA%B7%B8

kiwi.add_user_word("강경필", "NNG")
kiwi.tokenize(
    "헐 ㅋㅋㅋㅋ 이것도 되고 이건 되나욥 된다카이 강경필수업 강원대학교 재밌다 집에 놀러 간다 ㅋㅋdㅋㅋㅋ ㅎㅎㅎ NAACL",
    normalize_coda=True,
)

In [ ]:
from collections import Counter
from tqdm import tqdm

# 품사 목록(https://bab2min.github.io/kiwipiepy/v0.13.0/kr/#_9)를 참고해서 어떤 태그들을 사용하고 있는지 확인해 보세요.
permitted_tags = {"NNG", "NNP", "VV", "VA", "SL"}
counter = Counter()

# tqdm은 진행 상황을 보여주는 라이브러리입니다.
# mininterval=1은 1초 이상의 간격으로만 업데이트합니다. (너무 자주 업데이트해서 느려지는 것을 방지)
for news in tqdm(news_list, total=len(news_list), mininterval=1):
    body_text = news["text"].strip()
    if len(body_text) < 10:
        continue

    # 불용어 제거, 특정 품사이면서 길이가 2 이상인 단어만 추출
    tokens = [
        pos.form
        for pos in kiwi.tokenize(body_text, normalize_coda=True, stopwords=stopwords)
        if pos.tag in permitted_tags and len(pos.form) > 1
    ]
    counter.update(tokens)

# 상위 단어 빈도 출력
for word, count in counter.most_common(30):
    print(word, count)

In [ ]:
import matplotlib.font_manager as font_manager

# 폰트 경로 확인
font_list = font_manager.findSystemFonts(fontext="ttf")

print(font_list)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

most_freq = counter.most_common(100)
wc = WordCloud(
    font_path="/Users/kyeongpil/Library/Fonts/NanumGothic.ttf",  # 여기에 한글 지원되는 폰트 (나눔고딕) 경로를 넣어주세요
    prefer_horizontal=1.0,
    background_color="white",
)
cloud = wc.generate_from_frequencies(dict(most_freq))

plt.figure(figsize=(10, 8))
plt.imshow(cloud)
plt.show()

In [ ]:
# 다음과 같이 고빈도 단어를 기준으로 단어 사전을 구축할 수 있습니다.
vocab = [word for word, freq in counter.most_common() if freq >= 5]
print(f"단어 사전 크기: {len(vocab)}")

word2idx = {word: idx for idx, word in enumerate(vocab)}

print(f"\"강원대학교\"라는 단어의 index: {word2idx.get('강원대학교', -1)}")